In [13]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
import matplotlib.pyplot as plt
import regex as re
import nltk
import string
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.tag import pos_tag

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\stuti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df = pd.read_csv('polarity_subjectivity_data_without_LemmStemm.csv')
df.head(1)

,Unnamed: 0,URL,transcript,Comedian Name,word_count,Unique ID,neg_polarity,neu_polarity,pos_polarity,compound,subjectivity
0,0,https://scrapsfromtheloft.com/comedy/pete-holm...,\r\n \r\n [audience cheering and applauding] \...,Pete Holmes,10068,0,0.056,0.688,0.256,1.0,0.504545


In [5]:
# Removing punctuations from transcript
df['transcript'] = df['transcript'].str.replace(f'[{string.punctuation}]', '', regex = True)

# Removing special characters
df['transcript'] = df['transcript'].apply(lambda x: re.sub('[^\w\s]', '', x))

# Decapitalizing text
df['transcript'] = df['transcript'].apply(lambda x: x.lower())

In [6]:
df.head(1)

,Unnamed: 0,URL,transcript,Comedian Name,word_count,Unique ID,neg_polarity,neu_polarity,pos_polarity,compound,subjectivity
0,0,https://scrapsfromtheloft.com/comedy/pete-holm...,\r\n \r\n audience cheering and applauding \r\...,Pete Holmes,10068,0,0.056,0.688,0.256,1.0,0.504545


In [7]:
# Eliminating stop words
stop_words = stopwords.words('english')

df['tokens_lst'] = df['transcript'].apply(lambda x: [w for w in word_tokenize(x) if not w.lower() in stop_words])


In [8]:
df.head(1)

,Unnamed: 0,URL,transcript,Comedian Name,word_count,Unique ID,neg_polarity,neu_polarity,pos_polarity,compound,subjectivity,tokens_lst
0,0,https://scrapsfromtheloft.com/comedy/pete-holm...,\r\n \r\n audience cheering and applauding \r\...,Pete Holmes,10068,0,0.056,0.688,0.256,1.0,0.504545,"[audience, cheering, applauding, hello, hello,..."


In [9]:
# Counting most frequently ocurring words in the corpus
txt = df['tokens_lst'].apply(lambda x: ' '.join(x))
tokens = [w for lst in txt.apply(word_tokenize) for w in lst]
word_cnt = FreqDist(tokens)

In [10]:
len(word_cnt)

63855

In [15]:
# Map each word in list to adjective, nouns, verbs, adverbs
allowed_tag = ['NN', 'NNS', 'NNP', 'NNPS',     
               'RB', 'RBR', 'RBS', 
               'JJ', 'JJR', 'JJS',
               'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']

token_tag_lst = df['tokens_lst'].apply(lambda x: pos_tag(x))

In [16]:
token_tag_lst

0      [(audience, NN), (cheering, VBG), (applauding,...
1      [(im, NN), (funnier, NN), (told, VBD), (introd...
2      [(2024, CD), (netflix, JJ), (standup, NN), (co...
3      [(kevin, NN), (bridges, NNS), (overdue, JJ), (...
4      [(get, VB), (knees, NNS), (jacqueline, VB), (n...
                             ...                        
468    [(jammin, JJ), (new, JJ), (york, NN), (george,...
469    [(australian, JJ), (comedian, NN), (jim, NN), ...
470    [(hello, NN), (im, NN), (thomas, NN), (im, NN)...
471    [(complaints, NNS), (grievances, NNS), (hbo, V...
472    [(full, JJ), (transcript, NN), (bad, JJ), (ya,...
Name: tokens_lst, Length: 473, dtype: object

In [34]:
def filter_token(token_tag_lst):
    return [word for sub_lst in token_tag_lst for word, tag in sub_lst if tag in allowed_tag]

In [35]:
df['tag_tokens'] = df['tokens_lst'].apply(filter_token)

ValueError: not enough values to unpack (expected 2, got 1)